In [1]:
# Doesn't work
import os
import pandas as pd

lst = pd.DataFrame()

for root, dirs, files in os.walk('Users/Jenny/Desktop/brain-game/data'):
    for file in files:
        if file.endswith('.csv'):
            df = pd.read_csv(file)
            lst = lst.append(df)

In [8]:
csvs = [
        ['data/16-07-19/005_3(LEFT20s_REST20s_RIGHT20s)Trial1-2019-07-16-18-45-36.csv',
       'data/16-07-19/005_3(LEFT20s_REST20s_RIGHT20s)Trial2-2019-7-16-19-4-9.csv',
        ],
       ['data/16-07-19/006_3(LEFT20s_REST20s_RIGHT20s)Trial1-2019-7-16-19-38-15.csv',
       'data/16-07-19/006_3(LEFT20s_REST20s_RIGHT20s)Trial2-2019-7-16-19-44-13.csv',
       ],
       ['data/16-07-19/007_3(LEFT20s_REST20s_RIGHT20s)Trial1-2019-7-16-20-23-44.csv',
        'data/16-07-19/007_3(LEFT20s_REST20s_RIGHT20s)Trial2-2019-7-16-20-31-2.csv',
       ],
       ['data/18-07-19/008_trial1-2019-7-18-18-14-20.csv',
        'data/18-07-19/008_trial2-2019-7-18-18-21-22.csv',
       ],
        ['data/18-07-19/009_trial1-2019-7-18-18-54-0.csv',
        'data/18-07-19/009_trial2-2019-7-18-19-0-12.csv',
        'data/18-07-19/009_trial3-2019-7-18-19-5-44.csv',
        ],
        ['data/18-07-19/010_trial1-2019-7-18-19-40-9.csv'],
        ['data/18-07-19/011_trial1-2019-7-18-20-23-16.csv'],
    
        ['data/19-07-19/001-trial1-2019-7-19-19-16-45.csv',
         'data/19-07-19/001-trial2-2019-7-19-19-23-24.csv',
        ],
         ['data/19-07-19/012-trial1-2019-7-19-18-29-43.csv',
         'data/19-07-19/012-trial2-2019-7-19-18-37-28.csv'
        ]
      ]


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def load_data(csvs):
    lst = pd.DataFrame()
    for csv in csvs:
        for i in csv:
            df = pd.read_csv(i)
            lst = lst.append(df)
            
    lst = lst.drop(["Time","Channel 3", "Channel 4","Channel 5", "Channel 6"], axis=1)
    lst = lst[lst["Direction"] != "Rest"]
    lst.replace(["Left","Right"], [0,1], inplace=True)
    lst = lst.reset_index(drop=True)
    return lst

def load_individual(data):
    df = pd.DataFrame()
    for i in data:
        d = pd.read_csv(i)
        df = df.append(d)
        
    df = df.drop(["Time","Channel 3", "Channel 4","Channel 5", "Channel 6"], axis=1)
    df = df[df["Direction"] != "Rest"]
    df.replace(["Left","Right"], [0,1], inplace=True)
    df = df.reset_index(drop=True)
    return df

In [2]:
# FFT
from scipy.signal import butter, lfilter, iirfilter, welch
from scipy.fftpack import fft
import numpy.fft as fft1
from collections import Counter

def initial_preprocessing(raw_eeg_data,bp_lowcut =1, bp_highcut =70, bp_order=2,
                          notch_freq_Hz  = [60.0, 120.0], notch_order =2):
      
    nyq = 0.5 * 250 #Nyquist frequency
    low = bp_lowcut / nyq
    high = bp_highcut / nyq
       
    #Bandpass filter
    b_bandpass, a_bandpass = butter(bp_order, [low, high], btype='band')
    bp_filtered_eeg_data = np.apply_along_axis(lambda l: lfilter(b_bandpass, a_bandpass ,l),
                                                  0,raw_eeg_data)
    notch_filtered_eeg_data = bp_filtered_eeg_data
    low1  = notch_freq_Hz[0]
    high1 = notch_freq_Hz[1]
    low1  = low1/nyq
    high1 = high1/nyq
    b_notch, a_notch = iirfilter(2, [low1, high1], btype='bandstop')
    notch_filtered_eeg_data = np.apply_along_axis(lambda l: lfilter(b_notch, a_notch ,l),
                                                     0,bp_filtered_eeg_data)
           
    c3c4=[]
    for window in notch_filtered_eeg_data.T:
        f, Pxx_den = welch(window, 250)

        i = 0
        while i < len(f):
            if f[i]>8:
                    upper_in=i
                    break
            else:
                    i=i+1

        while i < len(f):
                if f[i]>12:
                    lower_in=i
                    break
                else:
                    i=i+1
    
        corrected_mean_spectra_PSDperBin = np.mean(Pxx_den[upper_in:lower_in])
        corrected_mean_spectram_PSDperBin = np.mean(Pxx_den)
        ratio=corrected_mean_spectra_PSDperBin/corrected_mean_spectram_PSDperBin
        c3c4.append(ratio)
        
    return c3c4

def process(data):
    p_train = list()
    for i in range(data.shape[0]):
        if i%500==0:
            if data[i-500:i].shape != (0, data.shape[1]):
                p_train.append(initial_preprocessing(data[i-500:i]))
    return p_train

def agg_label(data):
    p_train = list() 
    for i in range(data.shape[0]):
        if i%500==0:
            if len(data.iloc[i-500:i]) != 0:
                d = [x[0] for x in Counter(data.iloc[i-500:i].values).most_common(1)]
                p_train.append(d)
    return p_train

    #df = data[data.index % 50 == 0]
    #return df          

In [15]:
# Load all data 
data = load_data(csvs)
# Separate them into features and label
features = data.iloc[:,:4]
label = data.iloc[:,-1]
# Fast Fourier transform the features and labels
process_feature = process(features)
process_label = agg_label(label)

In [41]:
# Define training and testing data so that we leave one(individual) out
X_train = process(load_data(csvs[:-1]).iloc[:,:4])
X_test = process(load_individual(csvs[-1]).iloc[:,:4])
y_train = np.hstack(agg_label(load_data(csvs[:-1]).iloc[:,-1]))
y_test = np.hstack(agg_label(load_individual(csvs[-1]).iloc[:,-1]))

In [42]:
# Do grid search on Random Forest to find best parameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import accuracy_score


model = Pipeline([ ("normalizer", Normalizer()),
                    ("rfc", RandomForestClassifier(random_state=7)),
                   ])

parameters = {
    'rfc__n_estimators':[10,50,100,500],
    'rfc__max_features':["auto", "sqrt", "log2", None, 4]
}

clf = GridSearchCV(estimator=model, param_grid=parameters, cv=5, verbose=3)
clf.fit(X_train, y_train)
test_predict = clf.predict(X_test)
best_param = clf.best_params_
best_score = clf.best_score_
result = clf.cv_results_

# Print out best score, parameters
print(best_score, best_param)
#modle = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
#model.fit(X, y) 

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] rfc__max_features=auto, rfc__n_estimators=10 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=10, score=0.465, total=   0.0s
[CV] rfc__max_features=auto, rfc__n_estimators=10 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=10, score=0.518, total=   0.0s
[CV] rfc__max_features=auto, rfc__n_estimators=10 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=10, score=0.500, total=   0.0s
[CV] rfc__max_features=auto, rfc__n_estimators=10 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=10, score=0.412, total=   0.0s
[CV] rfc__max_features=auto, rfc__n_estimators=10 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=10, score=0.533, total=   0.0s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.522, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.540, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.469, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.469, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.489, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=auto, rfc__n_estimators=100, score=0.504, total=   0.3s
[CV] rfc__max_features=auto, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=auto, rfc__n_estimators=100, score=0.522, total=   0.3s
[CV] rfc__max_featu

[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.513, total=   0.2s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.544, total=   0.2s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.496, total=   0.2s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.487, total=   0.2s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.489, total=   0.2s
[CV] rfc__max_features=None, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=None, rfc__n_estimators=100, score=0.500, total=   0.4s
[CV] rfc__max_features=None, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=None, rfc__n_estimators=100, score=0.513, total=   0.4s
[CV] rfc__max_featu

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   55.8s finished


0.5128432240921169 {'rfc__max_features': None, 'rfc__n_estimators': 500}


In [145]:
# Load individual data
d = {}
for i in range(5,13):
    d["subject_{0}".format(i)] = load_individual(csvs[i-5])

In [146]:
# Define function add to "dict"
class my_dictionary(dict): 
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 

In [151]:
# Initialize best-score dictionary
best_scores  = my_dictionary()

In [152]:
# Random Forest model for induvidual
for i in range(5,13):
    
    model = Pipeline([  ("normalizer", Normalizer()),
                    ("rfc", RandomForestClassifier(random_state=7)),
                   ])

    parameters = {
    'rfc__n_estimators':[10,50,100,500],
    'rfc__max_features':["auto", "sqrt", "log2", None, 4]
    }

    clf = GridSearchCV(estimator=model, param_grid=parameters, cv=5, verbose=3)
    clf.fit(X_train, y_train)
    test_predict = clf.predict(X_test)
    best_param = clf.best_params_
    best_score = clf.best_score_
    result = clf.cv_results_

    print(best_score, best_param)
    best_scores.add("subject_"+str(i), best_score)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] rfc__max_features=auto, rfc__n_estimators=10 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=10, score=0.530, total=   0.0s
[CV] rfc__max_features=auto, rfc__n_estimators=10 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=10, score=0.662, total=   0.0s
[CV] rfc__max_features=auto, rfc__n_estimators=10 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=10, score=0.523, total=   0.0s
[CV] rfc__max_features=auto, rfc__n_estimators=10 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=10, score=0.523, total=   0.0s
[CV] rfc__max_features=auto, rfc__n_estimators=10 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=10, score=0.576, total=   0.0s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.523, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.623, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.517, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.530, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.583, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=auto, rfc__n_estimators=100, score=0.530, total=   0.2s
[CV] rfc__max_features=auto, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=auto, rfc__n_estimators=100, score=0.629, total=   0.2s
[CV] rfc__max_featu

[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.517, total=   0.1s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.675, total=   0.1s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.503, total=   0.1s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.543, total=   0.1s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.583, total=   0.1s
[CV] rfc__max_features=None, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=None, rfc__n_estimators=100, score=0.543, total=   0.2s
[CV] rfc__max_features=None, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=None, rfc__n_estimators=100, score=0.675, total=   0.2s
[CV] rfc__max_featu

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   35.1s finished


...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.523, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.623, total=   0.2s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.517, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.530, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=auto, rfc__n_estimators=50, score=0.583, total=   0.1s
[CV] rfc__max_features=auto, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=auto, rfc__n_estimators=100, score=0.530, total=   0.2s
[CV] rfc__max_features=auto, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=auto, rfc__n_estimators=100, score=0.629, total=   0.2s
[CV] rfc__max_featu

[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.517, total=   0.1s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.675, total=   0.1s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.503, total=   0.1s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.543, total=   0.1s
[CV] rfc__max_features=None, rfc__n_estimators=50 ....................
[CV]  rfc__max_features=None, rfc__n_estimators=50, score=0.583, total=   0.2s
[CV] rfc__max_features=None, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=None, rfc__n_estimators=100, score=0.543, total=   0.3s
[CV] rfc__max_features=None, rfc__n_estimators=100 ...................
[CV]  rfc__max_features=None, rfc__n_estimators=100, score=0.675, total=   0.2s
[CV] rfc__max_featu

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   36.3s finished


0.5814569536423841 {'rfc__max_features': None, 'rfc__n_estimators': 100}


In [153]:
# Print out best scores
best_scores

{'subject_5': 0.5814569536423841,
 'subject_6': 0.5814569536423841,
 'subject_7': 0.5814569536423841,
 'subject_8': 0.5814569536423841,
 'subject_9': 0.5814569536423841,
 'subject_10': 0.5814569536423841,
 'subject_11': 0.5814569536423841,
 'subject_12': 0.5814569536423841}

In [33]:
# Unpickle Laurence's model
import pickle
from sklearn.externals import joblib

model = joblib.load("ml/saved_rf_left/saved_model_rf_left_005.pkl") 

/Users/Jenny/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.19.1 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/Jenny/anaconda3/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.19.1 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [34]:
#s009 = load_individual(csvs[4])
#features = s009.iloc[:,:4]
#label = s009.iloc[:,-1]
#process_feature = process(features)
#process_label = agg_label(label)
#X_train, X_test, y_train, y_test = train_test_split(process_feature, np.hstack(process_label), test_size=0.2, random_state=42)

model.score(X_train, y_train)

-0.4383167375522088

In [ ]:
# Grid Search on SVM
from sklearn.preprocessing import Normalizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score

svm = Pipeline([    ("normalizer", Normalizer()),
                    ("svc", LinearSVC(penalty = 'l2', tol= 1e-2, random_state=7)),
                   ])

parameters = {
    'svc__C':[1e-5,1e-3,1,1e3,1e5,1e7,1e10,1e15]
}

gclf = GridSearchCV(estimator=svm, param_grid=parameters, cv=5, verbose=3)
gclf.fit(x_train_008, y_train_008)
test_predict_target = gclf.predict(x_test_008)
best_param = gclf.best_params_
best_score = gclf.best_score_
result = gclf.cv_results_

# Print out best score, parameters
print(best_score, best_param)